### Summary
SVM was used-> not as good results
- FastText
- Tfidf
- Personality
- Grammer
- VADER Sentiment

and their effect on accuracy in prediction


In [1]:
#importing libraries for NLP
import pandas as pd
import os as os
import json as js
import gzip
import numpy as np
import matplotlib.pyplot as plt

#Text Classification Material

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath
import gensim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#Image processing material
import imageio
from PIL import Image
from tqdm import tqdm
from scipy.special import expit as sigmoid
from scipy.stats import multivariate_normal

#Text Classification Material
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import scipy as sp

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MAHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
import os as os
import json as js
import gzip
import numpy as np
import matplotlib.pyplot as plt
import pickle

#importing libraries for NLP
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter

#Text Classification Material
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

#Text Classification Material

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MAHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MAHAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import gensim
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

from gensim.models import FastText

### Dataframe
Use the pickled full-feature dataframes

In [3]:
with open ('df_07_08', 'rb') as fr:
    df = pickle.load(fr)

In [4]:
df.shape
df2 = df.copy(deep=False)
df.shape,df2.shape

((18810, 68), (18810, 68))

In [5]:
def split_sentence(df):
    df["split"]=df["corpus"].apply(lambda x: x.split())
split_sentence(df)

In [6]:
sentences = list(df.split)

In [7]:
#takes about 10 min:
size=500
#Movie and TV Data
model_m = FastText(size=size, seed=1)  # instantiate
model_m.build_vocab(sentences=sentences)
model_m.train(sentences=sentences, total_examples=len(sentences), epochs=30)  # train

In [8]:
def sent_vectorizer(sent, model):
    '''Uses a pretrained FastText model to create an avage over 
    a review.'''
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw

In [9]:
def avg_sent(df,model,size):
    '''Takes a dataframe and adds features: Every dimension of
    a model becomes a column. Return new dataframe'''
    df["avg_sentence"]=df["split"].apply(lambda x: sent_vectorizer(x, model))
    length=df["avg_sentence"].apply(lambda x: len(x))
    df2=df[length==size]
    for i in range(size):
        if size==500:
            df2[str(i)] = df2["avg_sentence"].apply(lambda x : x[i])
        else:
            df2[str(1000+i)] = df2["avg_sentence"].apply(lambda x : x[i])
    return(df2)

In [10]:
# Adding both fasttext-500 and fasttext-1000 features to df's
x_m=avg_sent(df, model_m, 500)

C:\Users\MAHAM\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
# pickle full feature df's
with open ('df_07_08_fasttext', 'wb') as fw:
    pickle.dump(x_m, fw)

In [4]:
with open ('df_07_08_fasttext', 'rb') as fr:
    x_m = pickle.load(fr)

In [6]:
x_m.columns[:10]

Index(['reviewerID', 'asin', 'reviewText', 'overall', 'good', 'Comedy',
       'Art House & International', 'Drama', 'Action & Adventure', 'Horror'],
      dtype='object')

In [8]:
X=x_m["length"]
big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
X=np.c_[X,x_m[big5]]
X.shape

(18808, 6)

In [11]:

X = x_m["length"]
cv = CountVectorizer(max_features = 500, ngram_range=(1,2))
XX = cv.fit_transform(x_m["corpus"]).toarray()
# using relative word counts instead of absolute ones
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(XX).toarray()
# tfidf added
X = np.c_[tfidf,X] 
X.shape

(18808, 501)

In [18]:
big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
X=np.c_[X,df[big5]]
X.shape

(18810, 506)

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
X=np.array(df["sentiment"]).reshape(-1, 1)
y=df["good"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)
log = LogisticRegression(solver='liblinear')#regularization is applied by default
log.fit(X_train, y_train)
accuracy=log.score(X_test,y_test)

In [48]:
accuracy

0.7044125465178097

In [80]:
grammar=False
personality=False
interaction=False
size=500
if size==500:
    dimensions=list(range(size))
    embedding = [str(i) for i in dimensions]
else:
    dimensions=list(range(1000,1000+size))
    embedding = [str(i) for i in dimensions]

if personality==True:
    big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
    embedding = big5 + embedding

if grammar==True:
    grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
'adverb', 'preposition']
    embedding = grammatical + embedding

if interaction==True:
    interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']
    embedding = interactions + embedding
    
X=x_m[embedding]
X.shape

(18808, 500)

In [81]:
y=x_m["good"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)
log = LogisticRegression(solver='liblinear')#regularization is applied by default
log.fit(X_train, y_train)
accuracy=log.score(X_test,y_test)
accuracy

0.8290802764486975

In [88]:
personality=True
interaction=True
grammar=True
if personality==interaction==grammar==True:
    print("hi")
else:
    print("No")

hi


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

def prediction(df, fasttext=False, size=500,
               bow=False, top_words=500,
               personality=False,
               grammar=False, 
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,
               seed=1):
    '''This function return the accuracy of a prediction dependent on
    the set of features used.'''
    
    # Part I: Creating Feature Set
    if bow == True:
        # using n-grams as well
        cv = CountVectorizer(max_features = top_words, ngram_range=(1,2))
        XX = cv.fit_transform(df["corpus"]).toarray()
        # using relative word counts instead of absolute ones
        tfidf_transformer = TfidfTransformer()
        tfidf = tfidf_transformer.fit_transform(XX).toarray()
        # tfidf added
        X = tfidf


    if grammar == True:
        # adding Grammar features
        grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
        X=df[grammatical]

    if personality == True:
        # personality added
        big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
        X=df[big5]

  
    if interaction == True:
        # personality added
        interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']
                      
        X=df[interactions] 

    if personality==interaction==grammar==True:
        interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']        
        big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
        grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
        combined=interactions+big5+grammatical
        X=df[combined]

    if fasttext==True:
        if size==500:
            dimensions=list(range(size))
            embedding = [str(i) for i in dimensions]
        else:
            dimensions=list(range(1000,1000+size))
            embedding = [str(i) for i in dimensions]
            
        if personality==True:
            big5 = ["Extro", "Agree", "Neuro", "Open", "Consc"]
            embedding = big5 + embedding

            
        if grammar==True:
            grammatical = ["length","!","?",",","."," ",'noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition']
            embedding = grammatical + embedding
    
        if interaction==True:
            interactions=['Extro_Comedy','Extro_Art House & International', 'Extro_Drama','Extro_Action & Adventure', 'Extro_Horror', \
        'Extro_Science Fiction','Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International','Agree_Drama', \
        'Agree_Action & Adventure', 'Agree_Horror','Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',\
        'Neuro_Art House & International', 'Neuro_Drama','Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',\
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International','Open_Drama', 'Open_Action & Adventure', 'Open_Horror',\
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy','Consc_Art House & International', 'Consc_Drama',\
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction','Consc_Animation']
            embedding = interactions + embedding
            
        X=df[embedding]
        
        print(X.shape)
   
    if sentiment == True:
        X=np.array(df["sentiment"]).reshape(-1, 1)
  
    if bow_plus_personality == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['Extro', 'Agree', 'Neuro', 'Open', 'Consc']].values),
                          format='csr')
 
    if bow_plus_personality_plus_interactions == True:  
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['Extro_Comedy',
           'Extro_Art House & International', 'Extro_Drama',
           'Extro_Action & Adventure', 'Extro_Horror', 'Extro_Science Fiction',
           'Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International',
           'Agree_Drama', 'Agree_Action & Adventure', 'Agree_Horror',
           'Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',
           'Neuro_Art House & International', 'Neuro_Drama',
           'Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',
           'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International',
           'Open_Drama', 'Open_Action & Adventure', 'Open_Horror',
           'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy',
           'Consc_Art House & International', 'Consc_Drama',
           'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction',
           'Consc_Animation']].values),
                          format='csr')
        
    if bow_plus_personality_plus_grammar == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['length', '!', '?', ',', '.', ' ','noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition','Extro', 'Agree', 'Neuro', 'Open', 'Consc']].values),
                          format='csr')
        
    if bow_plus_personality_plus_interactions_plus_grammar == True:
        vectorizer = CountVectorizer(binary=True)
        X = sp.sparse.hstack((vectorizer.fit_transform(df.reviewText),
                      df[['length', '!', '?', ',', '.', ' ','noun', 'adj', 'verb (simple)',
       'verb (simple past)', 'verb (past participle)', 'verb (gerund)',
       'adverb', 'preposition','Extro', 'Agree', 'Neuro', 'Open', 'Consc', 'Extro_Comedy',
       'Extro_Art House & International', 'Extro_Drama',
       'Extro_Action & Adventure', 'Extro_Horror', 'Extro_Science Fiction',
       'Extro_Animation', 'Agree_Comedy', 'Agree_Art House & International',
       'Agree_Drama', 'Agree_Action & Adventure', 'Agree_Horror',
       'Agree_Science Fiction', 'Agree_Animation', 'Neuro_Comedy',
       'Neuro_Art House & International', 'Neuro_Drama',
       'Neuro_Action & Adventure', 'Neuro_Horror', 'Neuro_Science Fiction',
       'Neuro_Animation', 'Open_Comedy', 'Open_Art House & International',
       'Open_Drama', 'Open_Action & Adventure', 'Open_Horror',
       'Open_Science Fiction', 'Open_Animation', 'Consc_Comedy',
       'Consc_Art House & International', 'Consc_Drama',
       'Consc_Action & Adventure', 'Consc_Horror', 'Consc_Science Fiction',
       'Consc_Animation']].values),
                          format='csr')
    
    # Part II: Running a logit classification on the corresponding feature set X
    y=df["good"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)
    log = LogisticRegression(solver='liblinear')#regularization is applied by default
    log.fit(X_train, y_train)
    accuracy=log.score(X_test,y_test)
    #y_pred = log.predict(X_test)
    #cm = confusion_matrix(y_test, y_pred)
    #print(cm)
    
    return(accuracy)

Add word types and sentiment later on

In [13]:
columns=["Mean", "SD"]
tested=["Bag-of-Words_500",
        "Bag-of-Words_1000",
        "fastText",
        "Personality",
        "Grammar",
        "Sentiment",
        "Interaction",
        "Personality+Grammar+Interaction",
        "BOW+Personality",
        "BOW+Personality+Interactions",
        "BOW+Personality+Interactions+Grammar",
        "BOW+Personality+Grammar",
        "fasttext+Personality",
        "fasttext+Personality+Interactions+Grammar",
        "fasttext+Personality+Interactions",
        "fasttext+Personality+Grammar"
       ]
result_df=pd.DataFrame(columns=columns, index=tested)
result_df

,Mean,SD
Bag-of-Words_500,NaN,NaN
Bag-of-Words_1000,NaN,NaN
fastText,NaN,NaN
Personality,NaN,NaN
Grammar,NaN,NaN
Sentiment,NaN,NaN
Interaction,NaN,NaN
Personality+Grammar+Interaction,NaN,NaN
BOW+Personality,NaN,NaN
BOW+Personality+Interactions,NaN,NaN


In [14]:
import statistics

def create_results(results,n):
    '''This functions shows the mean accuracy over n samples.'''
    df=x_m
    
    # 1 BoW
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=500,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_500","Mean"] = mean
    results.loc["Bag-of-Words_500","SD"] = sd

    # 2 Tfidf-1000
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, bow=True,top_words=1000,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Bag-of-Words_1000","Mean"] = mean
    results.loc["Bag-of-Words_1000","SD"] = sd

    # 3 fasttext-500
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, fasttext=True,size=500,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["fastText","Mean"] = mean
    results.loc["fastText","SD"] = sd

    # 4 personality
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality","Mean"] = mean
    results.loc["Personality","SD"] = sd

 
    # 5 Personality genre interaction
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, interaction=True,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Interaction","Mean"] = mean
    results.loc["Interaction","SD"] = sd

    # 6 Grammar
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, grammar=True,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Grammar","Mean"] = mean
    results.loc["Grammar","SD"] = sd
    
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, personality=True,interaction=True,grammar=True,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Personality+Grammar+Interaction","Mean"] = mean
    results.loc["Personality+Grammar+Interaction","SD"] = sd    
    
    
    # 7 BOW_plus_personality
       
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality = True ,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["BOW+Personality","Mean"] = mean
    results.loc["BOW+Personality","SD"] = sd
    
     # 8 BOW_plus_personality_plus_interactions
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_interactions = True ,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Interactions","Mean"] = mean
    results.loc[ "BOW+Personality+Interactions","SD"] = sd
    
    # 9 BOW_plus_personality_plus_interactions_plus_grammar
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_grammar = True ,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Grammar","Mean"] = mean
    results.loc[ "BOW+Personality+Grammar","SD"] = sd
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, bow_plus_personality_plus_interactions_plus_grammar = True ,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "BOW+Personality+Interactions+Grammar","Mean"] = mean
    results.loc[ "BOW+Personality+Interactions+Grammar","SD"] = sd
    
    # 10 VADER sentiment
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, sentiment = True ,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "Sentiment","Mean"] = mean
    results.loc[ "Sentiment","SD"] = sd
    
    
    # 11 fasttext_plus_personality
       
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True , personality=True, size=500,
               bow=False, top_words=500,
               grammar=False, 
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["fasttext+Personality","Mean"] = mean
    results.loc["fasttext+Personality","SD"] = sd
    
     # 12 BOW_plus_personality_plus_interactions
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True, interaction=True,size=500,
               bow=False, top_words=500,
               grammar=False, 
               sentiment=False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Interactions","Mean"] = mean
    results.loc[ "fasttext+Personality+Interactions","SD"] = sd
    
    # 9 BOW_plus_personality_plus_interactions_plus_grammar
        
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True,grammar=True, size=500,
               bow=False, top_words=500,
               sentiment=False,
               interaction = False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Grammar","Mean"] = mean
    results.loc[ "fasttext+Personality+Grammar","SD"] = sd
    
    acc=[] 
    for seed in range(n):
        acc_n = prediction(df, fasttext = True ,personality=True,interaction=True, size=500,
               bow=False, top_words=500,
               grammar=True, 
               sentiment=False,
               bow_plus_personality = False,
               bow_plus_personality_plus_interactions = False,
               bow_plus_personality_plus_grammar=False,
               bow_plus_personality_plus_interactions_plus_grammar = False,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc[ "fasttext+Personality+Interactions+Grammar","Mean"] = mean
    results.loc[ "fasttext+Personality+Interactions+Grammar","SD"] = sd
        
    
    return(results)

In [15]:
resu=create_results(result_df,2)

(18808, 500)
(18808, 500)
(18808, 505)
(18808, 505)
(18808, 540)
(18808, 540)
(18808, 519)
(18808, 519)
(18808, 554)
(18808, 554)


In [16]:
resu

,Mean,SD
Bag-of-Words_500,0.813264,0.00206756
Bag-of-Words_1000,0.836124,0.00319533
fastText,0.830409,0.00300737
Personality,0.623206,0.00958598
Grammar,0.611244,0.00770637
Sentiment,0.703349,0.014285
Interaction,0.622807,0.0018796
Personality+Grammar+Interaction,0.673312,0.00150368
BOW+Personality,0.85513,0.00639065
BOW+Personality+Interactions,0.855263,0.00357125


In [69]:
resu

,Mean,SD
Bag-of-Words_500,0.813264,0.00206756
Bag-of-Words_1000,0.836124,0.00319533
fastText,0.830409,0.00300737
Personality,0.623206,0.00958598
Grammar,0.611244,0.00770637
Sentiment,0.703349,0.014285
Interaction,0.622807,0.0018796
BOW+Personality,0.85513,0.00639065
BOW+Personality+Interactions,0.855263,0.00357125
BOW+Personality+Interactions+Grammar,0.853535,0.00150368


In [ ]:
resu25 = create_results(result_df,25)
resu25

In [51]:
resu25.to_excel("results_07_08.xlsx")

,Mean,SD
Bag-of-Words_500,0.818905,0.0053227
Bag-of-Words_1000,0.84033,0.00486083
fastText,0.83512,0.00518655
Personality,0.62622,0.00786619
Grammar,0.562158,0.00701048
Sentiment,NaN,NaN
Interaction,0.622446,0.00522859
BOW+Personality,0.857682,0.00586806
BOW+Personality+Interactions,0.856534,0.00554962
BOW+Personality+Interactions+Grammar,0.857905,0.00554567


In [ ]:
#will be used for tomorrow
# adding word types and sentiment VADER
    # 8 Sentiment
    acc=[]
    for seed in range(n):
        acc_n=prediction(df, sentiment=True,seed=seed)
        acc.append(acc_n)
    mean = sum(acc)/len(acc)
    sd = statistics.stdev(acc)
    results.loc["Sentiment","Mean"] = mean
    results.loc["Sentiment","SD"] = sd
    
# with grammar
        acc=[]
        for seed in range(n):
            acc_n=prediction(df, tfidf=True,top_words=500,grammar=True,seed=seed)
            acc.append(acc_n)
        mean = sum(acc)/len(acc)
        sd = statistics.stdev(acc)
        results.loc["Tfidf+Grammar",area+": Mean"] = mean
        results.loc["Tfidf+Grammar",area+": SD"] = sd

        # with sentiment
        acc=[]
        for seed in range(n):
            acc_n=prediction(df, tfidf=True,top_words=500,sentiment=True,seed=seed)
            acc.append(acc_n)
        mean = sum(acc)/len(acc)
        sd = statistics.stdev(acc)
        results.loc["Tfidf+Sentiment",area+": Mean"] = mean
        results.loc["Tfidf+Sentiment",area+": SD"] = sd


        # Fasttext with combination
        acc=[]
        for seed in range(n):
            acc_n=prediction(df, fasttext=True,size=500,personality=True,seed=seed)
            acc.append(acc_n)
        mean = sum(acc)/len(acc)
        sd = statistics.stdev(acc)
        results.loc["fastText+Personality",area+": Mean"] = mean
        results.loc["fastText+Personality",area+": SD"] = sd
        
        acc=[]
        for seed in range(n):
            acc_n=prediction(df, fasttext=True,size=500,grammar=True,seed=seed)
            acc.append(acc_n)
        mean = sum(acc)/len(acc)
        sd = statistics.stdev(acc)
        results.loc["fastText+Grammar",area+": Mean"] = mean
        results.loc["fastText+Grammar",area+": SD"] = sd        
        
        acc=[]
        for seed in range(n):
            acc_n=prediction(df, fasttext=True,size=500,sentiment=True,seed=seed)
            acc.append(acc_n)
        mean = sum(acc)/len(acc)
        sd = statistics.stdev(acc)
        results.loc["fastText+Sentiment",area+": Mean"] = mean
        results.loc["fastText+Sentiment",area+": SD"] = sd
        